Conforme regramento específico, a tarefa será dividida da seguinte forma :

# Conteúdo
1. [Introdução](#1)
2. [Análise exploratória de dados - EDA](#2)
3. [Estratégia de validação](#3)
4. [Treino do modelo](#4)
5. [Submissão](#5)

Formatação realizada com ajuda de [StackOverflow](https://stackoverflow.com/questions/11948245/markdown-to-create-pages-and-table-of-contents).



## 1. Introdução <a name="1"></a>

Este trabalho constitui desafio vivencial da trilha de formação de cientista de dados na [PETROBRAS](https://petrobras.com.br).
Foi escolhida a competição de [Santander Customer Transaction Prediction](https://www.kaggle.com/c/santander-customer-transaction-prediction/overview). Essa é uma competição encerrada em Abril de 2019 que ofereceu US$ 65.000 do 1° ao 5° lugares. Nessa competição, o objetivo é prever quais clientes irão realizar uma transação específica no futuro, independente do valor.

In [ ]:
%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # gráficos

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%time
train_df= pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
test_df = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
sample_submission = pd.read_csv('../input/santander-customer-transaction-prediction/sample_submission.csv')
sample_submission.head()

## 2. Análise exploratória de dados - EDA <a name="2"></a>



[Santander EDA and Prediction](https://www.kaggle.com/gpreda/santander-eda-and-prediction)

[Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html)

Na descrição dos dados. é informado que os valores disponibilizados para previsão são numéricos e anonimizados, (conforme observado em [Santander Customer Transaction - EDA](https://www.kaggle.com/allunia/santander-customer-transaction-eda)) e tem estrutura similar à disponível para a solução do probelma, o que sugere que os dados são sintéticos.
Por se tratar de informação de transações financeiras, é compreensível que os dados sejam anonimizados, porém o fato de não saber o que cada uma das muitas colunas significa, dificulta a análise de dados e possíveis idéias de combinação de variáveis para melhorar o resultado do modelo.

In [ ]:
%time
train_df.describe()

É possível observar que as médias e desvios padrão estão distribuídos em uma grande variedade de valores, o que leva a entender que esses dados não estão padronizados, o que pode ser um problema para modelos baseados em redes neurais, porém é indiferente para modelos baseados em árvores de decisão, como o que será utilizado.

In [ ]:
%time
train_df.info(verbose = True, show_counts = True)

A partir da função info(), é possível observar que os dados de treino são compostos de um campo de texto "ID_code", um campo inteiro "target" (que pode assumir os valores de 0 ou 1) e todos os demais campos são do tipo número real. A partir dessa mesma função é possível observar que não existem valores faltantes (NaN).

In [ ]:
sns.countplot(x = train_df.target.values)

A partir da contagem da variável "target" é possível observar que existe um desequilíbrio de classes uma vez que existem muito mais transações não realizadas que transações realizadas.

In [ ]:
import matplotlib.pyplot as plt
def plot_feature_distribution(df1, df2, label1, label2, features):
    i = 0
    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(10,10,figsize=(18,44))

    for feature in features:
        i += 1
        plt.subplot(20,10,i)
        sns.kdeplot(df1[feature],
                    label=label1)
        sns.kdeplot(df2[feature],
                     label=label2)
        plt.xlabel(feature, fontsize=9)
        locs, labels = plt.xticks()
        plt.tick_params(axis='x', which='major', labelsize=6, pad=-6)
        plt.tick_params(axis='y', which='major', labelsize=6)
    plt.show();

In [ ]:
t0 = train_df.loc[train_df['target'] == 0]
t1 = train_df.loc[train_df['target'] == 1]
features = train_df.columns.values[2:202]
plot_feature_distribution(t0, t1, '0', '1', features)

É possível observar que as variáveis apresentam, via de regra, [distribuição normal](https://pt.wikipedia.org/wiki/Distribui%C3%A7%C3%A3o_normal) de dados.
É possível observar um relevante número de variáveis que apresentam distrinuições diferentes entre as transações realizadas (saída 1 corresponde às linhas cor de laranja) e as transações não realizadas (saida 0 corresponde a linhas azuis). Esse comportamento pode ser observado nas variáveis var_0, var_1, var_2, var_5, var_9, var_13, var_106, var_139 e muitas outras.

Algumas variáveis apresentam distribuições de frequência que se assemelham a [distribuição bimodal](https://pt.wikipedia.org/wiki/Distribui%C3%A7%C3%A3o_bimodal) como var_2, var_13, var_26, var_55, var_175, var_184, var_196.

In [ ]:
train_df.head()

In [ ]:
%time
features = train_df.columns.values[2:202]
plot_feature_distribution(train_df, test_df, 'train', 'test', features)

À primeira vista, as distribuições dos dados de treino e teste parecem idênticas. Uma avaliação minuciosa demonstra pequenas diferenças, especialmente próximo às modas, onde pequenas seções dos dados de treino (linha azul) ficam visíveis sob a distribuição dos dados de teste (linha laranja).
A existência de diferenças de distribuições entre os dados com saída 0 e saída 1 são um bom sinal pois demonstram que existem nos dados, informações que podem permitir a predição da variável objetivo (*target*).
A existência de uma grande semelhança entre os dados de treino e teste também é um bom sinal, pois evidencia que o teste é representativo do teste. Em caso contrário, o modelo poderia sofrer *overfitting* nos dados de treino e ter baixo desempenho nos dados de teste.
Mais do que felizes coincidências, as observações realizadas indicam (conforme observado no notebook [Santander Customer Transaction - EDA](https://www.kaggle.com/allunia/santander-customer-transaction-eda)) que os dados da competição são simulados, ou seja, foram gerados especificamente para a competição com métricas similares aos dados reais porém sem consistir em dados reais apenas anonimizados.
As informações a seguir foram adaptadas do notebook [Mutual Information](https://www.kaggle.com/ryanholbrook/mutual-information), que é parte do treinamento de *Feature Engineering*. Nesse desaafio, existe uma grande quantidade de variáveis de entrada e não é fornecida informação adicional sobre elas. Nessa situação, se faz pertinente a criação de uma métrica de utilidade das variáveis, uma função associando as variáveis de entrada com a variável de saída. A partir dessa métrica, é possível escolher um subconjunto das variáveis de entrada mais relacionadas com a variável de saída. A métrica utilizada será a de *Mutual Information*, que se assemelha bastante à correlação com a diferença de que, enquanto a correlação detecta relações lineares, e *Mutual Information* pode detectar qualquer tipo de relação.

In [ ]:
X = train_df.copy()
y = X.pop("target")
X.pop("ID_code")
discrete_features = X.dtypes == int
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

limite_linhas = 20000

mi_scores = make_mi_scores(X[::limite_linhas], y[::limite_linhas], discrete_features)
mi_scores[::3]  # show a few features with their MI scores

In [ ]:
top_features = mi_scores[:10].index.to_list()
top_features
top_features.append("target")
train_df[top_features]

## 3. Estratégia de validação <a name="3"></a>
Foi utilizada a estratégia padrão do Pycaret, que consiste em validação cruzada com 10 *folds*.

# Pycaret

A seguir, segue regressão realizada com base na [página do pycaret](https://pycaret.gitbook.io/docs/), mais especificamente no [Binary Classification Tutorial (CLF101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Binary%20Classification%20Tutorial%20Level%20Beginner%20-%20%20CLF101.ipynb). Primeiramente será realizada a instalação do Pycaret.

[TP2-Modelo_ML](https://www.kaggle.com/thomasandarilho/tp2-modelo-ml/edit/run/74270902)

In [ ]:
! pip install pycaret

Em seguida é feita a verificação da versão instalada da biblioteca.

In [ ]:
# check version
from pycaret.utils import version
version()

Feita a configuração do pycaret utilizando as 10 variáveis de entrada mais significativas conforme a informação mútua (MI score) de forma a otimizar o uso de recurso computacional, que é limitado no Kaggle (ao menos para a versão gratuita)

In [ ]:
from pycaret.classification import *
tp3 = setup(data = train_df[top_features],
            target = "target",
            session_id = 123,
            preprocess = False) 


Em seguida, é utilizado o pycaret para comparar o desempenho de diferentes modelos.
É possível observar que diversos modelos performam parecido em cerca de 0,9 de precisão.

In [ ]:
best_model = compare_models()

## 4. Treino do modelo <a name="4"></a>

O treino do modelo foi realizado com código obtido do notebook [922 in 3 minutes](https://www.kaggle.com/dott1718/922-in-3-minutes/notebook), conforme segue. Assim como a maioria das submissões com melhor classificação no ranking da competição, nesse código é utilizado um modelo do tipo [LightGBM](https://www.kaggle.com/prashant111/lightgbm-classifier-in-python).

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from scipy.special import logit

#train_df = pd.read_csv("../input/train.csv")
#test_df = pd.read_csv("../input/test.csv")
features = [x for x in train_df.columns if x.startswith("var")]

hist_df = pd.DataFrame()
for var in features:
    var_stats = train_df[var].append(test_df[var]).value_counts()
    hist_df[var] = pd.Series(test_df[var]).map(var_stats)
    hist_df[var] = hist_df[var] > 1

ind = hist_df.sum(axis=1) != 200
var_stats = {var:train_df[var].append(test_df[ind][var]).value_counts() for var in features}

pred = 0
for var in features:
    model = lgb.LGBMClassifier(**{
        'learning_rate':0.05, 'max_bin': 165, 'max_depth': 5, 'min_child_samples': 150,
        'min_child_weight': 0.1, 'min_split_gain': 0.0018, 'n_estimators': 41,
        'num_leaves': 6, 'reg_alpha': 2.0, 'reg_lambda': 2.54, 'objective': 'binary', 'n_jobs': -1})
    model = model.fit(np.hstack([train_df[var].values.reshape(-1,1),
                                 train_df[var].map(var_stats[var]).values.reshape(-1,1)]),
                               train_df["target"].values)
    pred += logit(model.predict_proba(np.hstack([test_df[var].values.reshape(-1,1),
                                 test_df[var].map(var_stats[var]).values.reshape(-1,1)]))[:,1])
    

## 5. Submissão <a name="5"></a>
O resultado do modelo é salvo em um arquivo .CSV entitulado "submission".

In [ ]:
## %time
pd.DataFrame({"ID_code":test_df["ID_code"], "target":pred}).to_csv("submission.csv", index=False)